import libraries

In [1]:
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Preparation of Data

In [3]:
# Define directory paths for infected and uninfected images
base_dir = 'D:/R001/cell_images'  # Update this with the actual base directory path

# Set up ImageDataGenerator for loading images in batches
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # Normalize images and set aside 20% for validation

# Load training data in batches
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,  # Adjust batch size as needed based on memory
    class_mode='binary',
    subset='training'
)

# Load validation data in batches
validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


Selecting Models

In [4]:
from tensorflow.keras.applications import ResNet50, InceptionV3, DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to add custom layers to the base model
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Load and modify the base models
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model_densenet = DenseNet201(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Create custom models
model_resnet = build_model(base_model_resnet)
model_inception = build_model(base_model_inception)
model_densenet = build_model(base_model_densenet)

Compile Models

In [4]:
# Compile the models
def compile_model(model):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

compile_model(model_resnet)
compile_model(model_inception)
compile_model(model_densenet)

print("Models defined and compiled")

Models defined and compiled


In [5]:
# Set the number of epochs (you can adjust this)
epochs = 5  # Increase as needed


ResNet (Training,Saving,Loading)

In [7]:
# Train ResNet model
print("Training ResNet model...")
model_resnet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Training ResNet model...
Epoch 1/5


C:\Users\Hamza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


689/689 ━━━━━━━━━━━━━━━━━━━━ 4691s 7s/step - accuracy: 0.9481 - loss: 0.1467 - val_accuracy: 0.5000 - val_loss: 0.8996
Epoch 2/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 5448s 8s/step - accuracy: 0.9753 - loss: 0.0697 - val_accuracy: 0.9517 - val_loss: 0.1539
Epoch 3/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 4689s 7s/step - accuracy: 0.9818 - loss: 0.0500 - val_accuracy: 0.9537 - val_loss: 0.1606
Epoch 4/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 5473s 8s/step - accuracy: 0.9860 - loss: 0.0373 - val_accuracy: 0.9548 - val_loss: 0.1966
Epoch 5/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 4767s 7s/step - accuracy: 0.9906 - loss: 0.0249 - val_accuracy: 0.9568 - val_loss: 0.1580


In [8]:
# Save the model in .keras format
model_resnet.save('model_resnet5ep.keras')

In [5]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model_resnet = load_model('model_resnet5ep.keras')

In [10]:
# Evaluate the loaded model on the validation data
loss, accuracy = loaded_model_resnet.evaluate(validation_generator)

# Print the accuracy
print(f'Accuracy of the loaded model: {accuracy * 100:.2f}%')

173/173 ━━━━━━━━━━━━━━━━━━━━ 219s 1s/step - accuracy: 0.9539 - loss: 0.1696
Accuracy of the loaded model: 95.68%


Inception (Training,Saving,Loading)

In [7]:
# Train InceptionV3 model
print("Training InceptionV3 model...")
model_inception.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Training InceptionV3 model...
Epoch 1/5


C:\Users\Hamza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


689/689 ━━━━━━━━━━━━━━━━━━━━ 2867s 4s/step - accuracy: 0.9260 - loss: 0.1954 - val_accuracy: 0.9539 - val_loss: 0.1435
Epoch 2/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 2178s 3s/step - accuracy: 0.9739 - loss: 0.0772 - val_accuracy: 0.9325 - val_loss: 0.2347
Epoch 3/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 2019s 3s/step - accuracy: 0.9824 - loss: 0.0495 - val_accuracy: 0.9426 - val_loss: 0.1758
Epoch 4/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 2501s 4s/step - accuracy: 0.9848 - loss: 0.0413 - val_accuracy: 0.9497 - val_loss: 0.2256
Epoch 5/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 2781s 4s/step - accuracy: 0.9865 - loss: 0.0372 - val_accuracy: 0.9452 - val_loss: 0.2021


In [8]:
# Save the model in .keras format
model_inception.save('model_inception5ep.keras')

In [6]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model_inception = load_model('model_inception5ep.keras')

In [11]:
# Evaluate the loaded model on the validation data
loss, accuracy = loaded_model_inception.evaluate(validation_generator)

# Print the accuracy
print(f'Accuracy of the loaded model: {accuracy * 100:.2f}%')

173/173 ━━━━━━━━━━━━━━━━━━━━ 171s 945ms/step - accuracy: 0.9468 - loss: 0.2004
Accuracy of the loaded model: 94.52%


DenseNet (Training,Saving,Loading)

In [5]:
# Train DenseNet model
print("Training DenseNet model...")
model_densenet.fit(
    train_generator,
    epochs=3,
    validation_data=validation_generator
)


Training DenseNet model...
Epoch 1/3


C:\Users\Hamza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


689/689 ━━━━━━━━━━━━━━━━━━━━ 7131s 10s/step - accuracy: 0.9448 - loss: 0.1555 - val_accuracy: 0.9557 - val_loss: 0.1520
Epoch 2/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 7167s 10s/step - accuracy: 0.9735 - loss: 0.0764 - val_accuracy: 0.9570 - val_loss: 0.1722
Epoch 3/3
689/689 ━━━━━━━━━━━━━━━━━━━━ 6784s 10s/step - accuracy: 0.9797 - loss: 0.0565 - val_accuracy: 0.9574 - val_loss: 0.1406


In [6]:
# Save the model in .keras format
model_densenet.save('model_densenet5ep.keras')

In [7]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model_densenet = load_model('model_densenet5ep.keras')

In [8]:
# Evaluate the loaded model on the validation data
loss, accuracy = loaded_model_densenet .evaluate(validation_generator)

# Print the accuracy
print(f'Accuracy of the loaded model: {accuracy * 100:.2f}%')

173/173 ━━━━━━━━━━━━━━━━━━━━ 436s 2s/step - accuracy: 0.9577 - loss: 0.1449
Accuracy of the loaded model: 95.74%


Final Results(through weighted average)

In [11]:
def predict_with_generator(model, generator):
    preds = []
    labels = []
    for batch, label in generator:
        pred = model.predict(batch)
        preds.extend(pred)
        labels.extend(label)
        if len(preds) >= generator.samples:
            break
    return np.array(preds)[:generator.samples], np.array(labels)[:generator.samples]

resnet_preds, y_test = predict_with_generator(loaded_model_resnet, validation_generator)
inception_preds, _ = predict_with_generator(loaded_model_inception, validation_generator)
densenet_preds, _ = predict_with_generator(loaded_model_densenet, validation_generator)

final_preds = (0.7 * resnet_preds + 0.6 * inception_preds + 0.5 * densenet_preds) / (0.7 + 0.6 + 0.5)
final_preds = (final_preds > 0.5).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, final_preds)
precision = precision_score(y_test, final_preds)
recall = recall_score(y_test, final_preds)
f1 = f1_score(y_test, final_preds)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7381125226860255
Precision: 0.725739848589126
Recall: 0.7655172413793103
F1 Score: 0.7450980392156863


Final Results(through stacking)

In [8]:
def predict_with_generator(model, generator):
    preds = []
    labels = []
    for batch, label in generator:
        pred = model.predict(batch)
        preds.extend(pred)
        labels.extend(label)
        if len(preds) >= generator.samples:
            break
    return np.array(preds)[:generator.samples], np.array(labels)[:generator.samples]


resnet_preds, y_test = predict_with_generator(loaded_model_resnet, validation_generator)
inception_preds, _ = predict_with_generator(loaded_model_inception, validation_generator)
densenet_preds, _ = predict_with_generator(loaded_model_densenet, validation_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

LogisticRegression

In [9]:
# Assuming you have already loaded the models and generated predictions (resnet_preds, inception_preds, densenet_preds)
# Stack the predictions together
stacked_predictions = np.column_stack((resnet_preds, inception_preds, densenet_preds))

# Train a Logistic Regression model as the meta-model
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, y_test)

# Make predictions with the meta-model
stacked_final_preds = meta_model.predict(stacked_predictions)

In [10]:
# Evaluate the performance of the stacked model
accuracy = accuracy_score(y_test, stacked_final_preds)
precision = precision_score(y_test, stacked_final_preds)
recall = recall_score(y_test, stacked_final_preds)
f1 = f1_score(y_test, stacked_final_preds)

In [11]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.9571687840290382
Precision: 0.9448957965383257
Recall: 0.9709618874773139
F1 Score: 0.9577515216612961


RandomForestClassifier

In [30]:
from sklearn.ensemble import RandomForestClassifier
# Assuming you have already loaded the models and generated predictions (resnet_preds, inception_preds, densenet_preds)
# Stack the predictions together
stacked_predictions = np.column_stack((resnet_preds, inception_preds, densenet_preds))

# Train a RF model as the meta-model
meta_model = RandomForestClassifier()
meta_model.fit(stacked_predictions, y_test)

# Make predictions with the meta-model
stacked_final_preds = meta_model.predict(stacked_predictions)

In [31]:
# Evaluate the performance of the stacked model
accuracy = accuracy_score(y_test, stacked_final_preds)
precision = precision_score(y_test, stacked_final_preds)
recall = recall_score(y_test, stacked_final_preds)
f1 = f1_score(y_test, stacked_final_preds)

In [32]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.997459165154265
Precision: 0.9963781238681637
Recall: 0.9985480943738657
F1 Score: 0.9974619289340102


SVM for Calssification

In [27]:
from sklearn.svm import SVC
# Assuming you have already loaded the models and generated predictions (resnet_preds, inception_preds, densenet_preds)
# Stack the predictions together
stacked_predictions = np.column_stack((resnet_preds, inception_preds, densenet_preds))

# Train a SVM C model as the meta-model
meta_model = SVC()
meta_model.fit(stacked_predictions, y_test)

# Make predictions with the meta-model
stacked_final_preds = meta_model.predict(stacked_predictions)

In [28]:
# Evaluate the performance of the stacked model
accuracy = accuracy_score(y_test, stacked_final_preds)
precision = precision_score(y_test, stacked_final_preds)
recall = recall_score(y_test, stacked_final_preds)
f1 = f1_score(y_test, stacked_final_preds)

In [29]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.9586206896551724
Precision: 0.9453648219950652
Recall: 0.973502722323049
F1 Score: 0.9592274678111588


Cascading

In [27]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to make predictions with each model and combine them at the end
def ensemble_predictions(models, data):
    predictions = []

    for model in models:
        preds = model.predict(data)
        predictions.append(preds)

    # Average the predictions or use any other ensemble method
    averaged_predictions = np.mean(predictions, axis=0)

    # Convert averaged predictions to binary (0 or 1)
    final_preds = (averaged_predictions > 0.5).astype(int).flatten()
    
    return final_preds

# Get the initial batch of data and labels from the validation generator
validation_generator.reset()  # Reset the generator
initial_batch, y_test = next(validation_generator)

# Perform ensemble prediction
final_preds = ensemble_predictions([loaded_model_resnet, loaded_model_inception, loaded_model_densenet], initial_batch)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 915ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [28]:
# Evaluate the performance
accuracy = accuracy_score(y_test, final_preds)
precision = precision_score(y_test, final_preds)
recall = recall_score(y_test, final_preds)
f1 = f1_score(y_test, final_preds)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.9375
Precision: 0.875
Recall: 1.0
F1 Score: 0.9333333333333333


Bagging

In [20]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to perform bagging
def bagging_predictions(models, data, n_estimators=10):
    predictions = []
    
    for i in range(n_estimators):
        # Bootstrap sampling: create a new dataset by sampling with replacement from the original data
        data_sample, _ = resample(data, data, replace=True)
        
        # Get predictions from each model
        preds = np.mean([model.predict(data_sample) for model in models], axis=0)
        predictions.append(preds)
    
    # Average the predictions across all estimators
    final_preds = np.mean(predictions, axis=0)
    
    # Convert averaged predictions to binary (0 or 1)
    final_preds = (final_preds > 0.5).astype(int).flatten()
    
    return final_preds

# Get the initial batch of data and labels from the validation generator
validation_generator.reset()  # Reset the generator
initial_batch, y_test = next(validation_generator)

# Perform bagging ensemble prediction
final_preds = bagging_predictions([loaded_model_resnet, loaded_model_inception, loaded_model_densenet], initial_batch, n_estimators=10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 935ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 963ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 999ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 923ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 896ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 929ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━

In [21]:
# Evaluate the performance
accuracy = accuracy_score(y_test, final_preds)
precision = precision_score(y_test, final_preds)
recall = recall_score(y_test, final_preds)
f1 = f1_score(y_test, final_preds)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.625
Precision: 0.6538461538461539
Recall: 0.85
F1 Score: 0.7391304347826086
